# Intro
I dette project vil jeg lave et objectorienteret ConvNet, der skal kunne genkende forskellige typer af tøj og sortere dem i kategorier.
Jeg har valgt at lave dem objektorienteret, da jeg før har forsøgt at lave ANN'er, hvor det var baseret hovedsagligt ud fra funktioner, hvilket ledte til, at de hele blev meget uoverskueligt og repeterivt.

# Importering af biblioteker
Først vil jeg importere nogle biblioteker med foruddefinerede funktioner, der kan hjælpe med at gøre udviklingsprocessen nemmere.

In [55]:
import numpy as np # Matricer og vektorer
np.random.seed(2) # For at kunne genskabe de samme resultater.

import pandas as pd # Til at læse data

In [56]:
debug = False # Skal der printes debug information?

# Defineringen af et lag
Først vil jeg definere en superklasse, lag, som de andre typer af lag skal nedarve fra.

In [57]:
class Lag:
    # __init__, også kendt som constructor, er kort for 'intialize', og 
    # er en metode, der kaldes, når et objekt af klassen Lag oprettes. 
    # Init har ikke noget output.
    def __init__(self) -> None:
        # Input- of outputtypen afhænger af, hvilekt slags lag det er. 
        # Men alle lag har in og out, så jeg definerer dem som None her, 
        # for at de er der, når jeg nedarver herfra.
        self.input = None
        self.output = None
    
    # I frem-metoden, udregner vi outputtet ud fra inputtet. Hvad der sker 
    # i frem(), afhænger af, hvilket slags lag det er.
    def frem(self, input):
        pass

    # I tilbage-metoden, udregner vi hvilke ændringer, der skal foretages 
    # på de trænede vægte og biaser, ud fra outputtet.
    def tilbage(self, pa_output): 
        # 'pa_output' er 'partielle afledte af outputtet'. Her snakkes der
        # om outputtet som blev udregnet i frem(). Dette er gives til laget,
        # fra det næste lag i netværkert under bagudpropagering.
        pass

    def update(self, skridt_længde):
        # Denne metode opdaterer vægtene og biaserne i laget, ud fra de 
        # ændringer, der blev udregnet i tilbage(). 

        # 'skridt_længde' er en hyperparameter, der bestemmer, hvor meget
        # vi vil ændre på vægtene og biaserne. Denne værdi er fastsat af mig
        # i dette tilfælde. Man kunne dog vælge at få den til at variere, alt
        # efter hvor langt netværket er fra sit mål.
        pass

# Definering af FF_Lag
Et FF-lag, eller et 'Fuldt forbundet' lag, er en type af lag, der ofte findes i Artificial Neural Networks. Her består et lag af neuroner, der er forbundet med synapser til alle de kommende lag. Her undlader jeg dog at bruge en aktiveringsfunktion, da dette kan gøres i et separat lag.

In [58]:
# Dette er et fuldt forbundet lag, også kendt som et fully connected layer, eller dense layer.
class FF(Lag):
    def __init__(self, n_input : int, n_output : int) -> None:
        if debug: print("FF init")

        # Her er n_input antallet af neuroner i det forrige lag, og n_output
        # er antallet af neuroner i det næste lag.
        # Dett skal jeg bruge for at vide, hvilket facon vægte og biaser skal have.
        
        self.w = np.random.randn(n_output, n_input).astype(np.float64)
        # w er vægtene. De er initialiseret til at være tilfældige tal. Vægte er
        # opbygget til at være samlet i en matrix, hvor højde er antaller af
        # neuroner i det næste lag, og bredde er antallet af neuroner i det forrige lag.
        # Vægte bliver ganget på inputtet i frem() metoden.
        
        self.b = np.random.randn(n_output, 1).astype(np.float64)
        # b er biaserne. De er initialiseret til at være tilfældige tal. Biaser er
        # opbygget til at være en vektor, hvor højden er antallet af neuroner
        # i det næste lag.
        # Biaser bliver lagt til vægtene i frem() metoden.

        self.pa_w = np.zeros_like(self.w).astype(np.float64)
        self.pa_b = np.zeros_like(self.b).astype(np.float64)
        # Disse variabler er sat til None, da de først bliver brugt i tilbage() og update() metoden.

        self.iterations = 0
        pass
    
    def frem(self, input) -> np.ndarray:
        if debug: print("FF frem")

        # Normalt ville man have en aktiveringsfunktion her, men dette kan også gøres
        # som et separat lag. Derfor er det udeladt her for at gøre det mere fleksibelt.

        vægtet = np.dot(self.w, input)
        # Inputtet ganges med vægtene, for at udregne outputtet. Dette gøres ved at
        # lave en matrix-transformation af inputvektoren.

        
        forskudt = vægtet + self.b
        # Biasvektoren lægges til det vægtede input. Dette kan forskyde 

        if debug:
            print("input")
            print(np.shape(input))
            print("vægtet")
            print(np.shape(vægtet))
            print("self.b")
            print(np.shape(self.b))
            print("forskudt")
            print(np.shape(forskudt))


        self.input = input
        self.output = forskudt
        # Inputtet gemmes, så det kan bruges i tilbage() metoden.

        return forskudt

    def tilbage(self, pa_output) -> np.ndarray:
        if debug: print("FF tilbage")
        
        self.pa_w += np.dot(pa_output, self.input.T)
        if debug: print("w clear")
        # Hvis man differencerer frem() metoden med hensyn til vægtene, finder man at
        # resultatet er inputtet ganget med partielle afledte af outputtet.
        # Vi transponerer inputtet, da det er en kolonnevektor, så resultatet bliver 
        # en matrix, der passer til vægtenes facon.

        self.pa_b += pa_output # * 1
        if debug: print("b clear")
        # Da biasene kun adderes på, er den partielle afledte af biaserne lig med 1.

        # Vi gemmer de partielle afledte, så de kan bruges i update() metoden. Grunden til at
        # vi ikke opdaterer vægtene og biaserne med det samme, er at vi gerne vil køre mini-batches.
        # Dette betyder at vi først vil køre en række input igennem, og derefter opdatere vægtene og biaserne.

        pa_input = np.dot(self.w.T, pa_output)
        if debug: print("input clear")
        # I denne metode har vi brugt pa_output fra det næste lag, til at udregne de ændringer,
        # vi skal lave til vægtene og biaserne. Vi skal også bruge et pa_output til det forrige lag.
        # Da inputtet ganges med vægtene, for at udregne outputtet, er den partielle afledte af inputtet
        # lig med vægtene ganget med pa_output fra det næste lag. Vi transponerer vægtene, for at få
        # de så matcher til outputtet.

        if debug:
            print("pa_output")
            print(np.shape(pa_output))
            print("pa_input")
            print(np.shape(pa_input))

        self.iterations += 1

        return pa_input
    
    def update(self, skridt_længde):
        if debug: print("FF update")

        self.w -= skridt_længde * (self.pa_w/self.iterations)
        self.b -= skridt_længde * (self.pa_b/self.iterations)
        # Vægtene og biaserne opdateres ved at trække den partielle afledte af vægtene
        # Da vi vil gå mod minimum, fremfor maksimum, trækker vi fra, og ganger
        # med skridtlængden.

        self.pa_w = np.zeros_like(self.w).astype(np.float64)
        self.pa_b = np.zeros_like(self.b).astype(np.float64)
        # Vi nulstiller de partielle afledte, så de er klar til næste Epoke.

        self.iterations = 0
        # Vi nulstiller antallet af iterationer, så vi kan tælle op til næste Epoke.
    

# Definering af Aktiveringsfunktionslag
Normal finder man aktiveringsfunktioner inde i neronerne, men man kan også vælge at lave dem til et separat lag. Så kan man holde kompleksiteten nede. Hellere flere simple lag, end få komplicerede.
<br>Desuden er det alstå også bare pænere at kigge på...

In [59]:
# Et lag, der bruger en aktiveringsfunktion. Dette lag har ingen vægte eller biaser.
class Funktion(Lag):
    def __init__(self, funktion, afledt_funktion) -> None:
        if debug: print("Funktion init")

        # Funktion er en aktiveringsfunktion, og afledt_funktion er sjovt nok den afledte funktionen.
        
        self.funktion = funktion
        self.afledt_funktion = afledt_funktion
        # Her gemmes funktionerne, så de kan bruges i frem() og tilbage() metoderne.
    
    def frem(self, input) -> np.ndarray:
        if debug: print("Funktion frem")
        
        self.input = input
        # Inputtet gemmes, så det kan bruges i tilbage() metoden.

        ikke_lineært = self.funktion(input)
        # Funktionen anvendes på inputtet, og outputtet returneres. Aktiveringsfunktioner bliver brugt
        # til at give netværkets beslutningsbariere en ikke-lineæritet, da ellers ville kunne blive
        # reduceret til en lineær transformation.
        # Dette er vigtigt, da mange problemer ikke er lineært separable.

        if debug: 
            print("input:")
            print(np.shape(input))

            print("output:")
            print(np.shape(ikke_lineært))

        return ikke_lineært
    
    def tilbage(self, pa_output) -> np.ndarray:
        if debug: print("Funktion tilbage")
        # Da aktiveringsfunktioner ikke har nogle vægte eller biaser, er der ikke noget at opdatere.

        pa_input = np.multiply(pa_output, self.afledt_funktion(self.input))
        # Her tager vi Hadamard-produktet af pa_output og den afledte funktion af inputtet.
        if debug: print(np.shape(pa_input))
        return pa_input
        pass

In [60]:
# For specifikke aktiveringsfunktioner, kan vi nedarve fra Funktion, og give dem en specifik funktion og afledt funktion.
# Her bruger jeg sigmoid, da den giver nogle pæne værdier mellem 0 og 1. Den er meget ligesom en tanH-funktion, der giver
# værdier mellem -1 og 1. Dog kan jeg godt lide den pæne form, som sigmoid har.
class Sigmoid(Funktion):
    def __init__(self):

        sigmoid = lambda x: 1 / (1 + np.exp(-np.clip(x, -500, 500)))
        afledt_sigmoid = lambda x: sigmoid(x) * (1 - sigmoid(x)) 
        # Her bruger jeg funktionelle lambda-udtryk, for at definere sigmoid og dens afledte funktion.
        # Dette gør, at jeg kan gemme dem i en variabel, og give dem til super's constructor.
        # Lamda-udtryk har det some regel også med at være lidt hurtigere end funktioner, men det
        # betyder ikke meget her, fordi jeg ikke fokuserer på hastighed.

        # I den afledte funktion, bruger jeg sigmoid(x) for at undgå at skulle udregne med for store tal.

        super().__init__(sigmoid, afledt_sigmoid)

# Udregning af netværkets tab
Når vi skal træne det neurale netværk, har vi brug for at vide, hvor godt netværket klarer sig. Derfor skal vi bruge et udtryk, der hedder 'tab'. Dette kan man finde ved at sammenligne netværkets output med éns ønskede output.
$$
Tab = 
(Output - ØnsketOutput)^2
$$
Grunden til at man sætter det i anden, er fordi programmet går efter mindst muligt tab. Hvis denne værdi kunne gå i negativ, fordi det ønskede output var højere end outputtet, ville programmet blive ved med at bevæge outputtet længere væk fra det ønskede output, fordi det ville blive en lavere værdi. Dertil får man den bonus, at tabet bliver eksponintielt voldsommere, jo mere forkert outputtet er.

In [61]:
# Ikke brugbar til træning, da man kun skal bruge den afledte funktion til at køre bagudpropageringen.
# Dog kan man bruge denne her til at holde øje med, om netværket lærer noget.
def tab(output, ønsket_output) -> float:
    # Tab er en funktion, der udregner hvor langt netværket er fra sit mål.
    return np.sum((output - ønsket_output) ** 2)

def pa_tab(output, ønsket_output) -> np.ndarray:
    # Den partielle afledte af tab-funktionen med hensyn til outputtet.
    if debug:
        print ("pa_tab")
        print(np.shape(output))
        print(np.shape(ønsket_output))
    return 2 * (output - ønsket_output)

# Definering af et Netværk
Normalt ville man ikek behøve at definere en klasse til at køre alle udregningerne, men da jeg gerne vi kunne lave flere versioner af forskellige netværk til tests med mere, er det hurtigere at samle det til en klasse, så jeg kan bruge den på forskellige måder, uden at skulle genskrive al koden.

In [62]:
class Netværk():
    def __init__(self, lag) -> None:
        self.lag = lag
        # Lag er en liste af lag, som netværket består af.

    def frem(self, input) -> np.ndarray:
        # Frem-metoden tager inputtet, og sender det igennem alle lagene i netværket.

        for l in self.lag:
            input = l.frem(input)
            # Inputtet sendes igennem hvert lag, og outputtet bliver det nye input.
        return input
    
    def __tilbage(self, pa_output) -> np.ndarray:
        # Tilbage-metoden tager partielle afledte af outputtet, og sender dem igennem alle lagene i netværket.

        for l in reversed(self.lag):
            pa_output = l.tilbage(pa_output)
            # Partielle afledte af outputtet sendes igennem hvert lag, og outputtet bliver det nye input.
        return pa_output
    
    def __update(self, skridt_længde) -> None:
        # Update-metoden opdaterer vægtene og biaserne i alle lagene i netværket.

        for l in self.lag:
            l.update(skridt_længde)
            # Vægtene og biaserne i hvert lag opdateres.
    
    def __tab(self, output, ønsket_output) -> float:
        # Tab-metoden tager outputtet og ønsket_outputtet, og udregner tabet.
        # Dette er udelukkende, så vi kan se, hvor langt netværket er fra sit mål.
        return tab(output, ønsket_output)
    

    # Jeg har valgt at lave de overstående metoder private, da de ikke skal bruges udenfor klassen.


    def træn(self, inputs : np.ndarray, ønskede_outputs : np.ndarray, skridt_længde : float, epoker : int, batchstørrelse : int, tab_print_interval : int) -> None:
        # Træn-metoden tager inputs, ønskede_outputs, skridt_længde, antallet af epoker og batchstørrelsen.
        # Den træner netværket, ved at køre inputs igennem netværket, udregne tabet, og køre bagudpropagering.
        # Dette gøres for et antal epoker, og med en batchstørrelse.
        
        sidste_tab = 0
        # Mini-batch gradientnedstigning.
        for i in range(epoker): 
            # For hver epoke

            for _ in range(batchstørrelse): 
                # For hver batch

                output = self.frem(inputs[i % len(inputs)])
                # Inputtet sendes igennem netværket, og outputtet returneres.

                pa_output = pa_tab(output, ønskede_outputs[i % len(ønskede_outputs)])
                # Den partielle afledte af tabet for outputtet udregnes.

                if debug: print(np.shape(pa_output))

                self.__tilbage(pa_output)
                # Partielle afledte sendes igennem netværket.
            
            self.__update(skridt_længde)
            # Efter at have kørt en batch igennem, opdateres vægtene og biaserne.


            
            if (i+1) % tab_print_interval == 0:
                tab = self.__tab(output, ønskede_outputs[i % len(ønskede_outputs)])
                print(f"Epoke {i+1}/{epoker} | {np.round(100*(i+1)/epoker,2)}% | Tab: {tab} | Ændring: {tab - sidste_tab}")
                # Her printes tabet, så vi kan se, om netværket lærer noget.

                sidste_tab = tab
        
        print("\nTræning færdig")
        print(f"Afsluttet med tab: {self.__tab(output, ønskede_outputs[i % len(ønskede_outputs)])}")


    def test(self,inputs : np.ndarray, ønskede_outputs : np.ndarray) -> None:

        rigtige_svar = 0

        for i in range(len(inputs)):

            output = self.frem(inputs[i])
            # Udregner outputtet for inputtet.

            max_index = np.argmax(output)
            output = np.zeros(output.shape)
            output[max_index] = 1
            # Finder det index, hvor outputtet er størst, og sætter det til 1, og resten til 0.

            if np.array_equal(output, ønskede_outputs[i]): rigtige_svar += 1

        præcision = np.round(rigtige_svar / len(inputs) * 100,2)
        print(f"Præcision: {præcision}%")




In [63]:
ann = Netværk([
    FF(2, 3), 
    Sigmoid(), 
    FF(3, 1), 
    Sigmoid()
    ])
# Her opretter jeg et netværk, som består af et inputlag med 2 neuroner, et skjult lag med 3 neuroner,
# et outputlag med 1 neuron.

inputs = np.reshape([[0, 0], [0, 1], [1, 0], [1, 1]], (4, 2, 1))
ønskede_outputs = np.reshape([[0], [1], [1], [0]], (4, 1, 1))
# Her opretter jeg inputs og ønskede_outputs, som er XOR-gate. Dette er et klassisk problem, som

#ann.træn(inputs, ønskede_outputs, 0.1, 10000, 4, 500)


# Test af ANN
Al den overstående kode er nok til at definere et ANN, så før jeg begynder at løse et stort datasæt, vil jeg lige teste om det fungerer først. Til dette bruger jeg den klassiske XOR test, der er et ikke-lineært separabelt problem.

Her man man se, at vi har opnået et relativt lavt tab. Nu vil jeg prøve at køre den igennem for at se dets resultater.

In [64]:
print(f"[0,0] : Expected 0 : Output {np.round(ann.frem(np.array([[0], [0]]))[0,0])}")
print(f"[1,0] : Expected 1 : Output {np.round(ann.frem(np.array([[1], [0]]))[0,0])}")
print(f"[0,1] : Expected 1 : Output {np.round(ann.frem(np.array([[0], [1]]))[0,0])}")
print(f"[1,1] : Expected 0 : Output {np.round(ann.frem(np.array([[1], [1]]))[0,0])}")

[0,0] : Expected 0 : Output 1.0
[1,0] : Expected 1 : Output 0.0
[0,1] : Expected 1 : Output 1.0
[1,1] : Expected 0 : Output 0.0


Den svarer rigtigt på opgaven.

# Importering af MNIST
MNIST er et stort dataset af håndskrevne cifre fra 0 - 9. Idéen her at at få netværket til at kunne genkende tallene og sortere dem ind i de rigtige kalsser. Men før dette kan gøres, skal vi importere denne data. I denne mappe har jeg to csv-filer med data. Den ene fil er til at træne netværker, mens den anden fil er til at teste dets præcision.

In [65]:
# Træningsdatet
train_data = pd.read_csv('mnist_train.csv')

train_labels = train_data.iloc[:, 0].to_numpy()
train_images = train_data.iloc[:, 1:].to_numpy().reshape(-1, 784, 1) / 255

# Testdataet
test_data = pd.read_csv('mnist_test.csv')

test_labels = test_data.iloc[:, 0].to_numpy()
test_images = test_data.iloc[:, 1:].to_numpy().reshape(-1, 784, 1) / 255

# Omdanner tallene i labels til vektorer med samme facon som outputtet fra netværket.
def format_labels(labels):
    formatted_labels = []
    for label in labels:
        formatted_label = np.zeros((10,1))
        formatted_label[label] = 1
        formatted_labels.append(formatted_label)
    return np.array(formatted_labels)

# Omdanner labels til vektorer
train_labels = format_labels(train_labels)
test_labels = format_labels(test_labels)


# Træning på MNIST
Nu er det bare tid til at træne et netværk på mnist. Alle billederne i MNIST er 1x28x28, så får det første lag skal have 784 neuroner. Selve sættet er heldigvis allerede sat i tilfældig rækkefølge, så det behøver vi ikke at bekymre os om.

In [66]:
mnistløser = Netværk([
    FF(784, 16),
    Sigmoid(),
    FF(16, 16),
    Sigmoid(),
    FF(16, 10),
    Sigmoid()
])

mnistløser.træn(train_images, train_labels, 0.01, 1280000, 10, 16000)

Epoke 16000/1280000 | 1.25% | Tab: 0.8681603631584202 | Ændring: 0.8681603631584202
Epoke 32000/1280000 | 2.5% | Tab: 0.9038726831247209 | Ændring: 0.035712319966300754
Epoke 48000/1280000 | 3.75% | Tab: 0.742167522642332 | Ændring: -0.16170516048238892
Epoke 64000/1280000 | 5.0% | Tab: 0.20919093435433173 | Ændring: -0.5329765882880002
Epoke 80000/1280000 | 6.25% | Tab: 0.9698798877904811 | Ændring: 0.7606889534361494
Epoke 96000/1280000 | 7.5% | Tab: 0.042209648047100166 | Ændring: -0.9276702397433809
Epoke 112000/1280000 | 8.75% | Tab: 0.02938781339104559 | Ændring: -0.012821834656054576
Epoke 128000/1280000 | 10.0% | Tab: 0.4469276166134054 | Ændring: 0.41753980322235984
Epoke 144000/1280000 | 11.25% | Tab: 0.5792614071537141 | Ændring: 0.1323337905403087
Epoke 160000/1280000 | 12.5% | Tab: 0.017602052320868065 | Ændring: -0.561659354832846
Epoke 176000/1280000 | 13.75% | Tab: 0.03961360396204589 | Ændring: 0.022011551641177827
Epoke 192000/1280000 | 15.0% | Tab: 0.1800515959998622

In [67]:
mnistløser.test(test_images, test_labels)

Præcision: 91.52%
